### 任务流程

1. 获得共同项目数据集(所有用户的，且独立到程序外的)（可选方案之一）
1. 先取得初始数据

3. 获得随机用户和随机评分项目
4. 修改目标项目评分
6. 计算平均评分
5. 获得共同项目数据集  (只获得目标用户相关即可)
7. 计算pearson系数   （只计算目标用户相关即可）
8. 找出近邻（需要一个最低相关度，或者一个topK）
9. 产生预测
12. 恢复数据
10. 重复3~8步，并记录数据值
11. 对照原始数据，计算评分

`存在问题：若随机获得的两用户相似度很低，又该如何`

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import random

In [2]:
data = pd.read_csv('./movie_lens/ratings.csv')
data = data.drop('timestamp',axis=1)
data.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
userId     100836 non-null int64
movieId    100836 non-null int64
rating     100836 non-null float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


#### 计算用户的平均评分

In [117]:
def average():
    rate_average = round( data.groupby(['userId'])['rating'].sum() / data.groupby(['userId'])['rating'].count() , 4)
    return rate_average

In [119]:
rate_average = average()
rate_average.head()

userId
1    4.3664
2    3.9483
3    2.4359
4    3.5556
5    3.6364
Name: rating, dtype: float64

#### 计算并获得共同评分项目数据集
``` userId_1 userId_2 movieIds
```

`抽出所有的共同评分的数据信息`

In [184]:
def common_file(user_number):
    f = open( 'common.txt', 'w', encoding='utf8')
    f.write('userId_1,userId_2,movieIds\n')
    for j in range(user_number):     #这里的迭代体 5 是用户数，假定用户1为比较用户，用户2为被比较用户，此处显然为用户1的个数
        if j ==0 :continue
        for i in range(user_number):    #这里的迭代体 11 是用户2的个数
            if i == 0 : continue
            if j == i : continue
                #获得电影编号
            movie_1 = data[data['userId']==j]
            movie_2 = data[data['userId']==i]
            f.write( str(j) +','+str(i)+',')
            for m_id in movie_1['movieId']:                   #遍历用户1的项目，每个都拿去在用户2的项目里检索
                judge = movie_2[movie_2['movieId'] == m_id]    #为获得共有电影编号参数 number 做准备
                number = judge['movieId']
                if judge.empty: pass                          #检查是否有共同项目，无则pass，有则写入文件
                else:  
                    f.write(str(int(number)) + ' ')
            f.write('\n')
    f.close()

In [ ]:
common_file(101)

#### 获得与目标用户有关的共同项目集

In [157]:
#制作公共影片文件  common.txt
def common_file(common_name,user_1,user_number):
    #获得公共影片信息
    #movie_1就是用户j评过分的电影,同理movie_2就是用户i评过分的电影
    f = open( common_name, 'w', encoding='utf8')
    f.write('userId_1,userId_2,movieIds\n')
    movie_1 = data[data['userId']==user_1]
    for user_2 in range(user_number):     #这里的迭代体 5 是用户数，假定用户1为比较用户，用户2为被比较用户，此处显然为用户1的个数
        if user_2 ==0 :continue
        if user_2 == user_1 : continue
                #获得电影编号
        movie_2 = data[data['userId']==user_2]
        f.write( str(user_1) +','+str(user_2)+',')
        for m_id in movie_1['movieId']:                   #遍历用户1的项目，每个都拿去在用户2的项目里检索
            judge = movie_2[movie_2['movieId'] == m_id]    #为获得共有电影编号参数 number 做准备
            Id = judge['movieId']
            if judge.empty: pass                          #检查是否有共同项目，无则pass，有则写入文件
            else:  
                f.write(str(int(Id)) + ' ')
        f.write('\n')
    f.close()

In [116]:
common_name = 'test2.txt'
common_file(common_name,5)

#### 取出共同评分项目数据集

In [120]:
def load_common(common_name): 
    common = pd.read_table(common_name,sep=',')
    common = common.fillna(0)
    return common

#### 获得共同项目中对应用户的评分

In [138]:
#生成pearson数值
#此函数无输出，但会生成文件:person.txt
def pearson_file(common,rate_average):
    
    '''
        common:共同评分项目的数据
        rate_average:各个用户的平均评分，索引为userId
        data:原始数据，用于获取评分值
    '''
        
    user = []
    length = len(common)   #存下总数据数
    person = []            #用来存储查看数据  [user1,user2,person]
    f = open( 'pearson.txt', 'w', encoding='utf8')
    f.write('user1,user2,pearson\n')
    for num in range(length):
        user1 = int(common.loc[num][0])          #拿到用户一的  id
        user2 = int(common.loc[num][1])
        if common.loc[num][2] == 0: continue     #当两用户无共同项目时，跳过
        user.append([user1,user2])               #记录当前计算用户id
        movie_common = common.loc[num][2].split()   #存储共同项目
        rating1 = 0
        rating2 = 0
        rating3 = 0
        for movie_id in movie_common:                 #对每个共同项目进行计算
            movie = data[data['movieId'] == int(movie_id)]
                                                         #拿到对应的个人评分
            rating_user1 = float( movie[movie['userId'] == user1]['rating'] ) 
            rating_user2 = float( movie[movie['userId'] == user2]['rating'] ) 
                                                         #开始进行累加的每一步
            rating1 += (rating_user1 - rate_average[user1])*(rating_user2 - rate_average[user2])
            rating2 += math.pow(rating_user1 - rate_average[user1],2)
            rating3 += math.pow(rating_user2 - rate_average[user2],2)
                                                        #计算最后的person得分
        rating = round( rating1/ ( math.sqrt(rating2) * math.sqrt(rating3) ) ,4)
        person.append([user1,user2,rating])
        f.write(str(user1)+','+str(user2)+','+str(rating)+'\n')
    f.close()

# pearson(common,rate_average,data)

In [131]:
def load_pearson():
    pearson = pd.read_table('pearson.txt',sep=',')
    return pearson

#### 计算并返回最近邻

In [189]:
def neighbour(pearson,user,movie,pearson_rate,pearson_number):
    pearson = pearson[pearson['user1'] == user]
    pearson = pearson[pearson['pearson'] > pearson_rate]
    neighbours = []
    for one in pearson['user2']:
        if verify(one,movie):
            neighbours.append(int(one))
        else:
            continue
    neighbours.sort(reverse = True)
    if pearson_number > 0:
        neighbours = neighbours[0:pearson_number]
    return neighbours

#### 验证该用户是否有目标项目的评分

In [188]:
def verify(user2,movie):
    user2 = data[data['userId']==user2]
    if user2.loc[[user2['movieId']==movie].index]['rating'] > 0:
        return True
    else:
        return False

#### 计算随机的movieId

In [106]:
def random_movie(user_random):
    res = data[data['userId']==user_random]
    res.reset_index(drop=True, inplace=True)
    movies = []
    for one in res['movieId']:
        movies.append(int(one))
    random.shuffle(movies)
    if movies[0] == 0: random_movie(user_random)
    else return movies[0]

#### 取得制造的随机数

In [109]:
def get_random(user_number):
    user_random = random.randint(1,user_number)
    movie_id = random_movie(user_random)
    
    return [user_random,movie_id]

#### 修改获得的随机用户的项目项目

In [110]:
def edit_rating(the_random,rating=0):
    user = the_random[0]
    movie_id = the_random[1]
    fake = data[data['userId']==user]
    y_rating = fake.loc[ fake[fake['movieId'] == movie_id].index ]['rating']
    fake.loc[ fake[fake['movieId'] == movie_id].index , 'rating']= rating
    data[data['userId']==user] = fake
    return y_rating

#### 恢复被修改的评分数值

In [162]:
def recovery(user,movie,y_rating):
    fake = data[data['userId']==user]
    fake.loc[ fake[fake['movieId'] == movie].index , 'rating']= y_rating
    data[data['userId']==user] = fake

In [128]:
the_random = get_random(user_number)
y_rating = float(edit_rating(the_random))
print(the_random)
print(y_rating)

[13, 597]
3.0


D:\anaconda\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


#### 预测函数

In [ ]:
def pridect(pearson,user,movie,rate_average,neighbours):
    user_1 = user
    rating_1 = 0
    rating_2 = 0
    y_pridect = 0
    for user_2 in neighbours:
        pearson_rating = float(pearson[pearson['user2']==user_1]['pearson'])
        rating_1 = pearson_rating * ()
        
    return y_pridect

In [181]:
float(pearson[pearson['user2']==49]['pearson'])

1.0

In [183]:
user2 = data[data['userId'] == 49]
user2 = user2[user2['movieId']==296]
user2

,userId,movieId,rating


In [27]:
#测试获得平均评分函数
rate_average = average(data)
rate_average.head()

userId
1    4.3664
2    3.9483
3    2.4359
4    3.5556
5    3.6364
Name: rating, dtype: float64

In [22]:
#测试获得随机用户和评分项目函数
the_random = get_random_movie(common,user_number)
the_random

[6, 3, 31]

In [20]:
#测试获得共同项目数据集文件函数
common_name = 'common.txt'
user_number = 10
common_file(data,common_name,user_number)
common = load_common()

In [ ]:
#测试pearson数据集文件获取函数
pearson(data,common,rate_average,data)

### test1
* 用户数定100，即user_number=100

In [174]:
user_number = 100 #设置最大用户数
user_number += 1
common_name = 'test1.txt'
pearson_rate = 0.5
pearson_number = 5  #最大近邻数，还未写



for time in range(11):
    
    the_random = get_random(user_number)      #拿到随机数据，形式为 [user_random,movie_random]
    user = the_random[0]
    movie = the_random[1]
    
    y_rating = float(edit_rating(the_random))    #修改并拿到随机项目的原始评分
    
    rate_average = average()                  #拿到平均评分

    common_file(common_name,user,user_number)      #写共同项目文件
    
    common = load_common(common_name)         #拿到共同项目数据
    
    pearson_file(common,rate_average)         #计算pearson系数，生成文件
    
    pearson = load_pearson()          #拿到pearson数值
    
    neighbours = neighbour(pearson,user,movie,pearson_rate,pearson_number)   #获得近邻列表,有两个可选参数，一个pearson_rate，一个pearson_number
    
    y_predict = pridect(pearson,user,movie,rate_average,neighbours)              #预测评分
    
    recovery(user,movie,y_rating)         #还原数据集，准备进行下一次循环
    
    
    
    
    
    print(the_random)
    print(y_rating)
    print(neighbours)
    print('*'*50)

D:\anaconda\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


[42, 436]
4.0
[35, 24, 14, 11]
**************************************************


D:\anaconda\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in double_scalars


[2, 48516]
4.0
[45, 43, 37, 33]
**************************************************
[28, 529]
3.5
[]
**************************************************
[25, 60069]
5.0
[49, 48, 47, 46]
**************************************************
[10, 72011]
2.5
[46, 43, 31, 29]
**************************************************
[6, 19]
2.0
[29, 25, 22, 10]
**************************************************
[32, 1079]
4.0
[50, 48, 41, 39]
**************************************************
[10, 7451]
3.5
[46, 43, 31, 29]
**************************************************
[18, 2421]
3.0
[35, 32, 26, 14]
**************************************************
[49, 2571]
4.5
[43, 37, 29, 26]
**************************************************
[26, 296]
4.0
[49, 38, 29, 19]
**************************************************
